In [2]:
# import libraries 
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List,Dict
from langchain.agents import AgentExecutor
import json
from langchain_core.agents import AgentActionMessageLog, AgentFinish
from abc import ABC, abstractmethod


In [3]:
# Setting environment variables 
load_dotenv()

True

In [93]:
class BaseGeneratorLLM(ABC):

    @abstractmethod
    def generate_content(user_resume:str,job_description:str):
        pass

In [94]:
# Generator LLM
class GeneratorLLM(BaseGeneratorLLM):

    def __init__(self,prompt_data:str,model_name:str = "gpt-3.5-turbo",temperature:float = 0.4):
        self.llm = ChatOpenAI(model=model_name, temperature=temperature)
        self.prompt_data : str = prompt_data
        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system",f"""You are an experienced resume writer with over 10 years of experience.
                            {prompt_data}"""),
                ("user", "User résumé: {user_resume} \n\n\n\n\n Job Description:{job_description}")
            ]
        )

    def generate_content(self,user_resume :str,job_description:str) -> str:
        self.chain = prompt | generator_llm
        response : json = self.chain.invoke({"user_resume":user_resume, "job_description":job_description})
        return response.content    
        

In [ ]:
class FactoryGeneratorLLM():

    def __init__(self,file_path: str):
        self.file_path = file_path
        self.prompts = json.load(open(file_path))
 
    def generate_object(self,category:str) -> BaseGeneratorLLM :

        if category == "Summary":
            prompt_data : str = self.prompts["Summary"]            
        else:
            raise Exception ("Category not supported")
        
        llm_agent : BaseGeneratorLLM = GeneratorLLM(prompt_data)
        return llm_agent

In [4]:
class AssessResponse(BaseModel):
    score: int = Field(description = "Score of the resume given based on scale of 1-10")
    feedback: List[str] = Field(description="List of feedback provided to improve the resume.")

In [ ]:
class AssessBaseLLM(ABC):
    
    @abstractmethod
    def access(self,prompt: str):
        pass

    def parse(self,output):
        # If no function was invoked, return to user
        if "function_call" not in output.additional_kwargs:
            return AgentFinish(return_values={"output": output.content}, log=output.content)
    
        # Parse out the function call
        function_call = output.additional_kwargs["function_call"]
        name = function_call["name"]
        inputs = json.loads(function_call["arguments"])
    
        # If the Response function was invoked, return to the user with the function inputs
        if name == "AssessResponse":
            return AgentFinish(return_values=inputs, log=str(function_call))
        # Otherwise, return an agent action
        else:
            return AgentActionMessageLog(
                tool=name, tool_input=inputs, log="", message_log=[output]
            )

In [ ]:
class AssessLLM(AssessBaseLLM):
    
    def __init__(self,prompt_data:str,model_name="gpt-3.5-turbo",temperature=0.0):
        self.llm = ChatOpenAI(model=model_name, temperature=temperature)
        self.prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",prompt_data,
                ),
                ("user", "User Section: {user_section} \n\n\n\n\n Job Description:{job_description}"),
            ]
        )
        self.llm_with_tools = self.llm.bind_functions([AssessResponse])
        self.agent = (
                {
                    "user_section": lambda x: x["user_section"],
                    "job_description": lambda x : x["job_description"],
                }
                | self.prompt
                | self.llm_with_tools
                | AssessBaseLLM.parse
        )
        self.agent_executor = AgentExecutor(tools = [],agent=self.agent, verbose=True)
        
    
    def access(self,user_section: str,job_description: str) -> AssessResponse:
        response = self.agent_executor.invoke({"job_description" : job_description,"user_section" :response.content },return_only_outputs=True)
        
        pass
        

In [43]:
# generator llm 
generator_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.4)

# TODO: add keywords 

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an experienced resume writer with over 10 years of experience.
            This are the introduction about summary section which you are going to rewrite for user who is applying for specific job.
            
            Choose a title for this section that reflects who user is and how he want to represent himself.
            
            Goal: This key section provides a concise overview of your qualifications, showing the employer, in a few short seconds, how user stand out from other candidates. You may draw qualifications from any area of user's life (e.g., work, volunteer experience, education, or other activities). Typically, include four to six (maximum seven) points outlining users relevant strengths and achievements, beginning with the most relevant to the job. Points may begin with nouns or adjectives. Describe users competitive advantage — the value user offer.
            
            Goal: Tailor Summary of Qualifications section (and résumé) according to Job Description. Keep this section precise and accurate wording.
            
            Goal: Summary section must be industry specific. However, rather than copy sentences from a Job Description, include key words commonly used in the industry (Job Description) to which you are applying. No need to include all the keywords just a few to keep it natural.
            
            Highlight any key or unique achievements that will help user stand out among other applicants. Use strong adjectives and facts to describe users strengths. A phrase such as “Two years’ experience completing projects in…” has more impact than “good knowledge of …” Include following bullet points:

            1. As a first bullet (if applicable), users experience (from paid/unpaid work, academics, or extracurricular activities) relevant to the position sought (e.g., one year experience in graphic design; three years process engineering experience with key responsibilities in product design and implementation; solid academic career focusing on business development initiatives in the field of specialty catalysts)
            2. Users relevant knowledge/skills/expertise (e.g., “computer proficiency, report writing, program planning, public speaking, problem-solving”)
            3. Any education that complements users practical experience (e.g., “machine design, resource assessment, marketing”)
            4. A general reference to where user developed the skill (e.g., “proven leadership skills developed through three summers as camp counsellor”). Include this level of detail only once or twice so that points do not become too lengthy
            5. Personal characteristics and attributes; however, include only those that are relevant to the position user is seeking and if it aligns with the job description (e.g., consistently able to deliver results under tight deadlines vs punctual, honest, etc.)
            6. Specialized training/education if completed by user otherwise it should not be mentioned (e.g., “CPR certification”)

            Rewrite just the user summary section along with job discription provided with all the above Goal in check and follow instructions provided in the above list if applicable.
            Your response should be in form of paragraph
            """,
        ),
        ("user", "User résumé: {user_resume} \n\n\n\n\n Job Description:{job_description}"),
    ]
)

In [44]:
chain = prompt | generator_llm

In [45]:
response = chain.invoke({"user_resume":user_resume, "job_description":job_description})

In [46]:
print(response.content)

Full-Stack Software Engineer with over 1.5 years of dynamic experience in Software Development, specializing in web development and AI initiatives. Proficient in problem-solving, machine learning, and algorithms, with a master’s degree in the field. Successfully enhanced ML pipeline performance, integrated backend to Azure, and optimized AI models resulting in significant cost savings and improved performance metrics. Skilled in API development, RESTful API interfaces, and database integration. Strong problem-solving skills, a passion for learning emerging technologies, and excellent communication abilities. Self-motivated, organized, and adept at working in fast-paced, collaborative environments. Proficient in Python and experienced with Azure Functions. Excited about the opportunity to contribute to innovative SaaS solutions in the capital markets industry.


In [48]:
response.content

'Full-Stack Software Engineer with over 1.5 years of dynamic experience in Software Development, specializing in web development and AI initiatives. Proficient in problem-solving, machine learning, and algorithms, with a master’s degree in the field. Successfully enhanced ML pipeline performance, integrated backend to Azure, and optimized AI models resulting in significant cost savings and improved performance metrics. Skilled in API development, RESTful API interfaces, and database integration. Strong problem-solving skills, a passion for learning emerging technologies, and excellent communication abilities. Self-motivated, organized, and adept at working in fast-paced, collaborative environments. Proficient in Python and experienced with Azure Functions. Excited about the opportunity to contribute to innovative SaaS solutions in the capital markets industry.'

In [86]:
# Acessing llm

class Response(BaseModel):
    score: int = Field(description = "Score of the resume given based on scale of 1-10")
    feedback: List[str] = Field(description="List of feedback provided to improve the resume.")


acessing_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.4)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an experienced recruiter writer with over 10 years of experience.
            This is Summary section of the resume rate it from scale of 1-10 and also provide some feedback to imporve the resume if available.
            Critize wherever possible
            """,
        ),
        ("user", "User Section: {user_section} \n\n\n\n\n Job Description:{job_description}"),
    ]
)

In [87]:
llm_with_tools = acessing_llm.bind_functions([Response])

In [88]:
def parse(output):
    # If no function was invoked, return to user
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(return_values={"output": output.content}, log=output.content)

    # Parse out the function call
    function_call = output.additional_kwargs["function_call"]
    name = function_call["name"]
    inputs = json.loads(function_call["arguments"])

    # If the Response function was invoked, return to the user with the function inputs
    if name == "Response":
        return AgentFinish(return_values=inputs, log=str(function_call))
    # Otherwise, return an agent action
    else:
        return AgentActionMessageLog(
            tool=name, tool_input=inputs, log="", message_log=[output]
        )

In [89]:
agent = (
    {
        "user_section": lambda x: x["user_section"],
        "job_description": lambda x : x["job_description"],
    }
    | prompt
    | llm_with_tools
    | parse
)

In [90]:
agent_executor = AgentExecutor(tools = [],agent=agent, verbose=True)

In [91]:
agent_executor.invoke({"job_description" : job_description,"user_section" :response.content },return_only_outputs=True)



> Entering new AgentExecutor chain...
{'arguments': '{"score":8,"feedback":["The summary provides a good overview of your experience, skills, and achievements in software development, particularly in web development and AI initiatives.","You have highlighted your key accomplishments such as enhancing ML pipeline performance, integrating backend to Azure, and optimizing AI models effectively.","You have mentioned your proficiency in Python and experience with Azure Functions, which are relevant to the job description provided."]}', 'name': 'Response'}

> Finished chain.


{'score': 8,
 'feedback': ['The summary provides a good overview of your experience, skills, and achievements in software development, particularly in web development and AI initiatives.',
  'You have highlighted your key accomplishments such as enhancing ML pipeline performance, integrating backend to Azure, and optimizing AI models effectively.',
  'You have mentioned your proficiency in Python and experience with Azure Functions, which are relevant to the job description provided.']}